In [1]:
import argparse
import math
import time
import torch
import torch.nn.functional as F
import torch.optim as optim
import logging
from datetime import datetime
import os
from torch.utils.data import Dataset, DataLoader
from os.path import join, exists
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from torch.nn import DataParallel
import transformers
import pickle
import sys
from sklearn.model_selection import train_test_split
from transformers import GPT2TokenizerFast, GPT2LMHeadModel, GPT2Config
from transformers import BertTokenizerFast
import pandas as pd
import torch.nn.utils.rnn as rnn_utils
import numpy as np

## 全部配置

In [3]:
class MyConfig():
    def __init__(self):
        self.device = '3'    #设置使用哪些显卡
        self.no_cuda = False    #不使用GPU进行训练
        self.vocab_path = '/kaggle/input/paworks/vocab.txt'    #词表路径
        self.model_config = '/kaggle/input/paworks/config.json'    #设置模型参数
        self.train_path = '/kaggle/input/paworks/train1.pkl'    #训练集路径
        self.max_len = 150    #训练时，输入数据的最大长度
        self.log_path = 'train.log'    #训练日志存放位置
        self.log = True    #是否记录日志
        self.ignore_index = -100    #对于ignore_index的label token不计算梯度
        self.epochs = 100    #训练的最大轮次
        self.batch_size = 8    #训练的batch size
        self.gpu0_bsz = 10    #0号卡的batch size
        self.lr = 2.6e-5    #学习率
        self.eps = 1.0e-09    #衰减率
        self.log_step = 1    #多少步汇报一次loss
        self.gradient_accumulation_steps = 4    #梯度积累
        self.max_grad_norm = 2.0    #参数的梯度范数
        self.save_model_path = 'model'    #模型输出路径
        self.pretrained_model = '/kaggle/input/paworks/pytorch_model.bin'    #预训练的模型的路径
        self.num_workers = 8    #dataloader加载数据时使用的线程数量
        self.patience = 0    #用于early stopping,设为0时,不进行early stopping.early stop得到的模型的生成效果不一定会更好。
        self.warmup_steps = 4000    #warm up步数
        self.val_num = 8    #验证集大小

# 代码主体

In [ ]:
class MyDataset(Dataset):
    """

    """

    def __init__(self, input_list, max_len):
        self.input_list = input_list
        self.max_len = max_len

    def __getitem__(self, index):
        input_ids = self.input_list[index]
        input_ids = input_ids[:self.max_len]
        input_ids = torch.tensor(input_ids, dtype=torch.long)
        return input_ids

    def __len__(self):
        return len(self.input_list)
    


def run():
    my_config = MyConfig()

    # 设置使用哪些显卡进行训练
    os.environ["CUDA_VISIBLE_DEVICES"] = my_config.device

    my_config.cuda = not my_config.no_cuda

    if my_config.batch_size < 2048 and my_config.warmup_steps <= 4000:
        print('[Warning] The warmup steps may be not enough.\n' \
              '(sz_b, warmup) = (2048, 4000) is the official setting.\n' \
              'Using smaller batch w/o longer warmup may cause ' \
              'the warmup stage ends with only little data trained.')

    # 创建日志对象
    logger = create_logger(my_config)
    # 当用户使用GPU,并且GPU可用时
    my_config.cuda = torch.cuda.is_available() and not my_config.no_cuda
    device = 'cuda:0' if my_config.cuda else 'cpu'
    my_config.device = device
    logger.info('using device:{}'.format(device))

    # 初始化tokenizer
    tokenizer = BertTokenizerFast(vocab_file=my_config.vocab_path, sep_token="[SEP]", pad_token="[PAD]", cls_token="[CLS]")
    my_config.sep_id = tokenizer.sep_token_id
    my_config.pad_id = tokenizer.pad_token_id
    my_config.cls_id = tokenizer.cls_token_id

    # 创建模型的输出目录
    if not os.path.exists(my_config.save_model_path):
        os.mkdir(my_config.save_model_path)

    # 创建模型
    if my_config.pretrained_model:  # 加载预训练模型
        print(f'model_config_path:{my_config.model_config}  model:{my_config.pretrained_model}')
        model_config = GPT2Config.from_json_file(my_config.model_config)
        model = GPT2LMHeadModel.from_pretrained(my_config.pretrained_model, config=model_config)
    else:  # 初始化模型
        model_config = GPT2Config.from_json_file(my_config.model_config)
        model = GPT2LMHeadModel(config=model_config)
    model = model.to(device)
    logger.info('model config:\n{}'.format(model.config.to_json_string()))
    assert model.config.vocab_size == tokenizer.vocab_size

    # 并行训练模型
    if my_config.cuda and torch.cuda.device_count() > 1:
        model = DataParallel(model).cuda()
        # model = BalancedDataParallel(my_config.gpu0_bsz, model, dim=0).cuda()
        logger.info("use GPU {} to train".format(my_config.device))

    # 计算模型参数数量
    num_parameters = 0
    parameters = model.parameters()
    for parameter in parameters:
        num_parameters += parameter.numel()
    logger.info('number of model parameters: {}'.format(num_parameters))

    # 记录参数设置
    logger.info("my_config:{}".format(my_config))

    # 加载训练集和验证集
    # ========= Loading Dataset ========= #
    train_dataset, validate_dataset = load_dataset(logger, my_config)

    train(model, logger, train_dataset, validate_dataset, my_config)
    
def create_logger(my_config):
    """
    将日志输出到日志文件和控制台
    """
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)

    formatter = logging.Formatter(
        '%(asctime)s - %(levelname)s - %(message)s')

    # 创建一个handler，用于写入日志文件
    file_handler = logging.FileHandler(
        filename=my_config.log_path)
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.INFO)
    logger.addHandler(file_handler)

    # 创建一个handler，用于将日志输出到控制台
    console = logging.StreamHandler()
    console.setLevel(logging.DEBUG)
    console.setFormatter(formatter)
    logger.addHandler(console)

    return logger    

def load_dataset(logger, my_config):
    """
    加载训练集和验证集
    """
    logger.info("loading training dataset and validating dataset")
    train_path = my_config.train_path

    with open(train_path, "rb") as f:
        input_list = pickle.load(f)

    # 划分训练集与验证集
    val_num = my_config.val_num
    input_list_train = input_list[val_num:]
    input_list_val = input_list[:val_num]
    # test
    # input_list_train = input_list_train[:24]
    # input_list_val = input_list_val[:24]
    print(f'train {len(input_list_train)} {input_list_train}')
    print(f'valid {len(input_list_val)} {input_list_val}')
    train_dataset = MyDataset(input_list_train, my_config.max_len)
    val_dataset = MyDataset(input_list_val, my_config.max_len)


    return train_dataset, val_dataset

def train_epoch(model, train_dataloader, optimizer, scheduler, logger,
                epoch, my_config):
    model.train()
    device = my_config.device
    # pad_id = my_config.pad_id
    # sep_id = my_config.sep_id
    ignore_index = my_config.ignore_index
    epoch_start_time = datetime.now()
    total_loss = 0  # 记录下整个epoch的loss的总和

    # epoch_correct_num:每个epoch中,output预测正确的word的数量
    # epoch_total_num: 每个epoch中,output预测的word的总数量
    epoch_correct_num, epoch_total_num = 0, 0

    for batch_idx, (input_ids, labels) in enumerate(train_dataloader):
        # 捕获cuda out of memory exception
        try:
            input_ids = input_ids.to(device)
            labels = labels.to(device)
            outputs = model.forward(input_ids, labels=labels)
            logits = outputs.logits
            loss = outputs.loss
            loss = loss.mean()

            # 统计该batch的预测token的正确数与总数
            batch_correct_num, batch_total_num = calculate_acc(logits, labels, ignore_index=ignore_index)
            # 统计该epoch的预测token的正确数与总数
            epoch_correct_num += batch_correct_num
            epoch_total_num += batch_total_num
            # 计算该batch的accuracy
            batch_acc = batch_correct_num / batch_total_num

            total_loss += loss.item()
            if my_config.gradient_accumulation_steps > 1:
                loss = loss / my_config.gradient_accumulation_steps

            loss.backward()
            # 梯度裁剪
            torch.nn.utils.clip_grad_norm_(model.parameters(), my_config.max_grad_norm)

            # 进行一定step的梯度累计之后，更新参数
            if (batch_idx + 1) % my_config.gradient_accumulation_steps == 0:
                # 更新参数
                optimizer.step()
                # 更新学习率
                scheduler.step()
                # 清空梯度信息
                optimizer.zero_grad()

            if (batch_idx + 1) % my_config.log_step == 0:
                logger.info(
                    "batch {} of epoch {}, loss {}, batch_acc {}, lr {}".format(
                        batch_idx + 1, epoch + 1, loss.item() * my_config.gradient_accumulation_steps, batch_acc,
                        scheduler.get_lr()))

            del input_ids, outputs

        except RuntimeError as exception:
            if "out of memory" in str(exception):
                logger.info("WARNING: ran out of memory")
                if hasattr(torch.cuda, 'empty_cache'):
                    torch.cuda.empty_cache()
            else:
                logger.info(str(exception))
                raise exception

    # 记录当前epoch的平均loss与accuracy
    epoch_mean_loss = total_loss / len(train_dataloader)
    epoch_mean_acc = epoch_correct_num / epoch_total_num
    logger.info(
        "epoch {}: loss {}, predict_acc {}".format(epoch + 1, epoch_mean_loss, epoch_mean_acc))

    # save model
    logger.info('saving model for epoch {}'.format(epoch + 1))
    model_path = join(my_config.save_model_path, 'epoch{}'.format(epoch + 1))
    if not os.path.exists(model_path):
        os.mkdir(model_path)
    model_to_save = model.module if hasattr(model, 'module') else model
    model_to_save.save_pretrained(model_path)
    logger.info('epoch {} finished'.format(epoch + 1))
    epoch_finish_time = datetime.now()
    logger.info('time for one epoch: {}'.format(epoch_finish_time - epoch_start_time))

    return epoch_mean_loss


def validate_epoch(model, validate_dataloader, logger, epoch, my_config):
    logger.info(f"start validating {len(validate_dataloader)}")
    model.eval()
    device = my_config.device
    # pad_id = my_config.pad_id
    # sep_id = my_config.sep_id
    ignore_index = my_config.ignore_index
    epoch_start_time = datetime.now()
    total_loss = 0
    # 捕获cuda out of memory exception
    try:
        with torch.no_grad():
            for batch_idx, (input_ids, labels) in enumerate(validate_dataloader):
                input_ids = input_ids.to(device)
                labels = labels.to(device)
                outputs = model.forward(input_ids, labels=labels)
                logits = outputs.logits
                loss = outputs.loss
                loss = loss.mean()

                total_loss += loss.item()
                del input_ids, outputs

            # 记录当前epoch的平均loss
            epoch_mean_loss = total_loss / len(validate_dataloader)
            logger.info(
                "validate epoch {}: loss {}".format(epoch + 1, epoch_mean_loss))
            epoch_finish_time = datetime.now()
            logger.info('time for validating one epoch: {}'.format(epoch_finish_time - epoch_start_time))
            return epoch_mean_loss
    except RuntimeError as exception:
        if "out of memory" in str(exception):
            logger.info("WARNING: ran out of memory")
            if hasattr(torch.cuda, 'empty_cache'):
                torch.cuda.empty_cache()
        else:
            logger.info(str(exception))
            raise exception


def train(model, logger, train_dataset, validate_dataset, my_config):
    train_dataloader = DataLoader(
        train_dataset, batch_size=my_config.batch_size, shuffle=True, num_workers=my_config.num_workers, collate_fn=collate_fn,
        drop_last=True
    )
    validate_dataloader = DataLoader(validate_dataset, batch_size=my_config.batch_size, shuffle=True,
                                     num_workers=my_config.num_workers, collate_fn=collate_fn, drop_last=True)
#     early_stopping = EarlyStopping(my_config.patience, verbose=True, save_path=my_config.save_model_path)
    t_total = len(train_dataloader) // my_config.gradient_accumulation_steps * my_config.epochs
    optimizer = transformers.AdamW(model.parameters(), lr=my_config.lr, eps=my_config.eps)
    # scheduler = transformers.WarmupLinearSchedule(optimizer, warmup_steps=my_config.warmup_steps, t_total=t_total)
    scheduler = transformers.get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=my_config.warmup_steps, num_training_steps=t_total
    )

    logger.info('starting training')

    # 用于记录每个epoch训练和验证的loss
    train_losses, validate_losses = [], []
    # 记录验证集的最小loss
    best_val_loss = 10000
    # 开始训练
    for epoch in range(my_config.epochs):
        # ========== train ========== #
        train_loss = train_epoch(
            model=model, train_dataloader=train_dataloader,
            optimizer=optimizer, scheduler=scheduler,
            logger=logger, epoch=epoch, my_config=my_config)
        train_losses.append(train_loss)

        # ========== validate ========== #
        validate_loss = validate_epoch(
            model=model, validate_dataloader=validate_dataloader,
            logger=logger, epoch=epoch, my_config=my_config)
        validate_losses.append(validate_loss)

        # 保存当前困惑度最低的模型，困惑度低，模型的生成效果不一定会越好
        if validate_loss < best_val_loss:
            best_val_loss = validate_loss
            logger.info('saving current best model for epoch {}'.format(epoch + 1))
            model_path = join(my_config.save_model_path, 'min_ppl_model'.format(epoch + 1))
            if not os.path.exists(model_path):
                os.mkdir(model_path)
            model_to_save = model.module if hasattr(model, 'module') else model
            model_to_save.save_pretrained(model_path)

        #  如果patience=0,则不进行early stopping
        if my_config.patience == 0:
            continue
        # 这里不知道为何无法导入EarlyStopping
#         early_stopping(validate_loss, model)
#         if early_stopping.early_stop:
#             logger.info("Early stopping")
#             break
    logger.info('training finished')
    logger.info("train_losses:{}".format(train_losses))
    logger.info("validate_losses:{}".format(validate_losses))


def caculate_loss(logit, target, pad_idx, smoothing=True):
    if smoothing:
        logit = logit[..., :-1, :].contiguous().view(-1, logit.size(2))
        target = target[..., 1:].contiguous().view(-1)

        eps = 0.1
        n_class = logit.size(-1)

        one_hot = torch.zeros_like(logit).scatter(1, target.view(-1, 1), 1)
        one_hot = one_hot * (1 - eps) + (1 - one_hot) * eps / (n_class - 1)
        log_prb = F.log_softmax(logit, dim=1)

        non_pad_mask = target.ne(pad_idx)
        loss = -(one_hot * log_prb).sum(dim=1)
        loss = loss.masked_select(non_pad_mask).mean()  # average later
    else:
        # loss = F.cross_entropy(predict_logit, target, ignore_index=pad_idx)
        logit = logit[..., :-1, :].contiguous().view(-1, logit.size(-1))
        labels = target[..., 1:].contiguous().view(-1)
        loss = F.cross_entropy(logit, labels, ignore_index=pad_idx)
    return loss


def calculate_acc(logit, labels, ignore_index=-100):
    logit = logit[..., :-1, :].contiguous().view(-1, logit.size(-1))
    labels = labels[..., 1:].contiguous().view(-1)

    _, logit = logit.max(dim=-1)  # 对于每条数据，返回最大的index
    # 进行非运算，返回一个tensor，若labels的第i个位置为pad_id，则置为0，否则为1
    non_pad_mask = labels.ne(ignore_index)
    n_correct = logit.eq(labels).masked_select(non_pad_mask).sum().item()
    n_word = non_pad_mask.sum().item()
    return n_correct, n_word

def collate_fn(batch):
    input_ids = rnn_utils.pad_sequence(batch, batch_first=True, padding_value=0)
    labels = rnn_utils.pad_sequence(batch, batch_first=True, padding_value=-100)
    return input_ids, labels
run()

2022-12-20 03:05:31,189 - INFO - using device:cpu
2022-12-20 03:05:31,189 - INFO - using device:cpu
2022-12-20 03:05:31,189 - INFO - using device:cpu
2022-12-20 03:05:31,189 - INFO - using device:cpu
2022-12-20 03:05:31,189 - INFO - using device:cpu
2022-12-20 03:05:31,189 - INFO - using device:cpu


[Warning] The warmup steps may be not enough.
(sz_b, warmup) = (2048, 4000) is the official setting.
Using smaller batch w/o longer warmup may cause the warmup stage ends with only little data trained.
model_config_path:/kaggle/input/paworks/config.json  model:/kaggle/input/paworks/pytorch_model.bin


2022-12-20 03:05:32,285 - INFO - model config:
{
  "_name_or_path": "/kaggle/input/paworks/pytorch_model.bin",
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 300,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 10,
  "n_positions": 300,
  "output_past": true,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "transformers_version": "4.20.1",
  "use_cache": true,
  "vocab_size": 13317
}

2022-12-20 03:05:32,285 - INFO - model config:
{
  "_name_or_path": "/kaggle/input/paworks/pytorch_

train 57 [[101, 784, 720, 3221, 924, 7372, 782, 8043, 102, 924, 7372, 782, 2218, 3221, 680, 2832, 924, 782, 6370, 4989, 924, 7372, 1394, 1398, 8024, 2400, 2902, 4212, 1394, 1398, 5276, 2137, 2824, 2857, 6608, 985, 2772, 5314, 802, 924, 7372, 7032, 6569, 818, 4638, 924, 7372, 1062, 1385, 511, 102], [101, 924, 7372, 782, 1377, 809, 3221, 6443, 8043, 102, 3791, 782, 8038, 898, 4212, 517, 924, 7372, 3791, 518, 6392, 4989, 4638, 924, 7372, 1062, 1385, 809, 1350, 3791, 2526, 510, 6121, 3124, 3791, 6226, 6226, 2137, 4638, 1071, 800, 924, 7372, 5299, 5302, 511, 102, 5632, 4197, 782, 8038, 2208, 3144, 1744, 2157, 1038, 6387, 809, 702, 782, 6716, 819, 5307, 5852, 924, 7372, 689, 1218, 8024, 1963, 5739, 1744, 1227, 1394, 4852, 511, 2769, 1744, 3257, 3198, 679, 1038, 6387, 511, 102], [101, 924, 7372, 782, 3300, 784, 720, 721, 1218, 8043, 102, 6370, 5276, 6432, 3209, 8038, 1403, 2832, 924, 782, 6237, 7025, 924, 7372, 1394, 1398, 3340, 3621, 511, 102, 7599, 7372, 2824, 2857, 510, 924, 7372, 7032, 53

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
2022-12-20 03:05:38,331 - INFO - starting training
2022-12-20 03:05:38,331 - INFO - starting training
2022-12-20 03:05:38,331 - INFO - starting training
2022-12-20 03:05:38,331 - INFO - starting training
2022-12-20 03:05:38,331 - INFO - starting training
2022-12-20 03:05:38,331 - INFO - starting training
/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker numbe